In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

local_llm = "llama3"

urls = [
    "https://mormonr.org/qnas/a9l1T/the_kinderhook_plates",
    "https://www.fairlatterdaysaints.org/answers/Kinderhook_Plates",
    "https://rsc.byu.edu/no-weapon-shall-prosper/did-joseph-smith-translate-kinderhook-plates",
]

docs = [FireCrawlLoader(api_key="fc-d616f0b9d1b44e6485daa619b6ac1bfe", url=url, mode="scrape").load() for url in urls]

# Split documents
# docs_list = [item for sublist in docs for item in sublist]
docs_list = []
for sublist in docs:
    for item in sublist:
        docs_list.append(item)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap = 20,
)

docs_splits = text_splitter.split_documents(docs_list)

# Filter out complex metadata and ensure proper document formatting
filtered_docs = []
for doc in docs_splits:
    # Ensure the doc is an instance of Document and has a 'metadata' attribute
    if isinstance(doc, Document) and hasattr(doc, "metadata"):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_docs.append(Document(doc.page_content, metadata=clean_metadata))


# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=filtered_docs,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever

Exception: Failed to scrape URL. Status code: 429